link da aula: https://nbviewer.org/github/logictensornetworks/logictensornetworks/blob/master/tutorials/1-grounding_non_logical_symbols.ipynb

In [2]:
import ltn
import tensorflow as tf
import numpy as np

Constantes

In [3]:
c1 = ltn.Constant([2.1,3], trainable=False)
c2 = ltn.Constant([[4.2,3,2.5],[4,-1.3,1.8]], trainable=False)
c3 = ltn.Constant([0.,0.], trainable=True)

In [4]:
print(c1)
print(c1.tensor)
print(c3)
print(c3.tensor)

ltn.Constant(tensor=[2.1 3. ], trainable=False, free_vars=[])
tf.Tensor([2.1 3. ], shape=(2,), dtype=float32)
ltn.Constant(tensor=<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>, trainable=True, free_vars=[])
<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>


Predicatos

In [5]:
mu = tf.constant([2.,3.])
P1 = ltn.Predicate.Lambda(lambda x: tf.exp(-tf.norm(x-mu,axis=1)))

class ModelP2(tf.keras.Model):
    """For more info on how to use tf.keras.Model:
    https://www.tensorflow.org/api_docs/python/tf/keras/Model"""
    def __init__(self):
        super(ModelP2, self).__init__()
        self.dense1 = tf.keras.layers.Dense(5, activation=tf.nn.elu)
        self.dense2 = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid) # returns one value in [0,1]
    def call(self, x):
        x = self.dense1(x)
        return self.dense2(x)

modelP2 = ModelP2()
P2 = ltn.Predicate(modelP2)

In [6]:
c1 = ltn.Constant([2.1,3],trainable=False)
c2 = ltn.Constant([4.5,0.8],trainable=False)

print(P1(c1))
print(P2(c1))

ltn.Formula(tensor=0.9048374891281128, free_vars=[])
ltn.Formula(tensor=0.5473829507827759, free_vars=[])


Predcatos com varias entradas

In [7]:
class ModelP3(tf.keras.Model):
    def __init__(self):
        super(ModelP3, self).__init__()
        self.dense1 = tf.keras.layers.Dense(5, activation=tf.nn.elu)
        self.dense2 = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid) # returns one value in [0,1]

    def call(self, inputs):
        x1, x2 = inputs[0], inputs[1] # multiple arguments are passed as a list
        x = tf.concat([x1,x2],axis=1) # axis=0 is the batch axis
        x = self.dense1(x)
        return self.dense2(x)

P3 = ltn.Predicate(ModelP3())
print(P3([c1,c2])) # multiple arguments are passed as a list

ltn.Formula(tensor=0.2072950303554535, free_vars=[])


Funções

In [17]:
f1 = ltn.Function.Lambda(lambda args: args[0]-args[1])

class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(5)
    def call(self, x):
        x = self.dense1(x)
        return self.dense2(x)

model_f2 = MyModel()
f2 = ltn.Function(model_f2)

In [18]:
c1 = ltn.Constant([2.1,3], trainable=False)
c2 = ltn.Constant([4.5,0.8], trainable=False)
print(f1([c1,c2])) # multiple arguments are passed as a list
print(f2(c1))

ltn.Term(tensor=[-2.4  2.2], free_vars=[])
ltn.Term(tensor=[-0.10879046  1.1963565  -1.2033721   0.02323772 -1.0980843 ], free_vars=[])


In [19]:
print(f2(P3([c1,c2])))

TypeError: The input to a LTN Function should be instances of <class 'ltn.core.Term'>. Got an instance of <class 'ltn.core.Formula'> instead.

Variaveis

faz a rotulação da varivel dos dados dado,
No exemplo abaixo temos -> x = ltn.Variable('x',np.random.normal(0.,1.,(10,2)))
Ele vai armazenar na variavel "X" uma sequencia de numeros aleatorios em forma de matriz 10x2

In [25]:
x = ltn.Variable('x',np.random.normal(0.,1.,(10,2)))
y = ltn.Variable('y',np.random.normal(0.,4.,(5,2)))

print(x)

ltn.Variable(label=x, tensor=[[ 0.7346985   0.65531373]
 [ 1.443111    1.6075578 ]
 [ 0.32270494 -0.26803637]
 [ 1.6647294  -0.5585481 ]
 [-1.1140903  -0.04859015]
 [-1.724438   -1.4043763 ]
 [ 0.9942487   0.69943327]
 [ 0.7251135  -1.5805838 ]
 [ 0.09729848  0.3634398 ]
 [ 0.05521763  0.22182561]], free_vars=['x'])


O codigo abaixo usar o predicato 3(que aceita mais de 1 entrada)

In [28]:
# Notice that the outcome is a 2 dimensional tensor where each cell
# represents the satisfiability of P3 with each individual in x and in y.
res1 = P3([x,y])
print(res1)
print(res1.take('x',2).take('y',0)) # gives the result calculated with the 3rd individual in x and the 1st individual in y
#nesse print acima ele ta pegando na linha 3 o valor da coluna 1

ltn.Formula(tensor=[[0.99920523 0.60689783 0.8370218  0.4459729  0.6495899 ]
 [0.9990158  0.5498914  0.8189362  0.3727268  0.61040914]
 [0.999444   0.7187543  0.8756635  0.6030621  0.732201  ]
 [0.9997559  0.88506705 0.9438298  0.82265717 0.876102  ]
 [0.9988156  0.50589055 0.77536696 0.35597456 0.5549041 ]
 [0.99937797 0.70662946 0.856322   0.6035395  0.70898175]
 [0.9992733  0.62930137 0.8478121  0.47210744 0.6687765 ]
 [0.9998232  0.9234639  0.9606038  0.88114506 0.91533333]
 [0.99910736 0.58020264 0.8207615  0.42106676 0.6239452 ]
 [0.99916106 0.59874326 0.82809305 0.4449879  0.63806903]], free_vars=['x', 'y'])
ltn.Formula(tensor=0.9994440078735352, free_vars=[])


No codigo abaixo vamos usar a funcao 1(f1), dados os argumentos (x,y). Lembrando que f1, faz a subtração de x-y, no caso para cada elemento de x e y

In [29]:
# This is also valid with the outputs of `ltn.Function`
res2 = f1([x,y])
print(res2.tensor.shape)
print(res2.free_vars)
print(res2.take('x',2).take('y',0)) # gives the result calculated with the 3rd individual in x and the 1st individual in y

(10, 5, 2)
['x', 'y']
ltn.Term(tensor=[10.046727  2.39023 ], free_vars=[])


In [33]:
res3 = P3([c1,y])
print(res3) # Notice that no axis is associated to a constant.

ltn.Formula(tensor=[0.9984635  0.4786462  0.7827325  0.30179757 0.5453604 ], free_vars=['y'])


Variaveis feitas de constante treinaveis

In [34]:
c1 = ltn.Constant([2.1,3], trainable=True)
c2 = ltn.Constant([4.5,0.8], trainable=True)

with tf.GradientTape() as tape:
    # Notice that the assignation must be done within a tf.GradientTape.
    # Tensorflow will keep track of the gradients between c1/c2 and x.
    # Read tutorial 3 for more details.
    x = ltn.Variable.from_constants("x", [c1,c2], tape=tape)
    res = P2(x)
tape.gradient(res.tensor,c1.tensor).numpy() # the tape keeps track of gradients between P2(x), x and c1

array([ 0.10783625, -0.05137738], dtype=float32)